# Лабораторная работа №4

ФИО:   
Группа: 

Отправлять можно следующими способами:
1. Запушить этот ноутбук в GitHub в репозиторий, где у вас лежат ноутбуки с лабами

Deadlines:
- Занятие №7 в семестре

Что необходимо сделать:  
- Обучить различные модели глубокого обучения на имеющихся данных  

---
## Читайте задание внимательно

Исходные данные:
1. В [табличке](https://docs.google.com/spreadsheets/d/1NOE0D4JQgD6LbvUqWboUI1TFj4P87ugbqUTDquxlGEI/edit?usp=sharing) необходимо узнать название своего датасета 
2. Скачать нужны вам данные можно в [Google Drive](https://drive.google.com/drive/folders/1sbsjBsJ_ln0XgXCI9R6s17pvyvApgcwF?usp=sharing)
  
---
Теперь по пунктам, что я от вас жду:  
1. Загрузить необходимые данные к себе и считать (read) их в переменную.
2. Понять, у вас задача классификации (бинарной или многоклассовой) или регрессии.
3. Сделать предобработку данных:  
     1. Разделить выборку на тренировочную (train) и тестовую (test). _Обратите внимание, что обучать скейлеры и определять, какими значениями вы будете заполнять пропуски, вы будете на train выборке, а применять и на train, и на test_.
     2. Проверить пропуски в данных. Если они есть, заполнить одной из стратегий, предложенных в ноутбуке для семинара №3. P.S. Для численных и категориальных переменных будут разные стратегии.
     3. Отнормировать численные переменные (`StandardScaler`, `MinMaxScaler`).
     4. Закодировать категориальные признаки по одной из стратегий.
4. Оформить данные в виде класса `Dataset` из библиотеки `torch` (как мы это делали на семинаре), а затем засунуть в `Dataloader` (тоже делали на семинаре).
5. Обучить на тренировочном множестве:
     1. Очень простую однослойную нейросеть с оптимизатором `SGD` ([link](https://pytorch.org/docs/stable/optim.html)).
     2. Нейросеть посложнее (с 1 скрытым слоем) с оптимизатором `Adam` ([link](https://pytorch.org/docs/stable/optim.html)).
     3. Нейросеть еще сложнее (с 3+ скрытыми слоями) с оптимизатором `Adam` ([link](https://pytorch.org/docs/stable/optim.html)).
6. Посчитайте loss на train и test множествах, в зависимости от эпохи обучения. Провизуализируйте это с помощью библиотеки `matplotlib` (выйдет так называемая **learning curve**, кривая обучения модели).
6. Посчитайте метрики на train и test множествах:
     1. Для задачи классификации -- Accuracy
     2. Для задачи регрессии -- MAE
7. Сравните метрики относительно train/test, так и относительно разных моделей. Ответьте на следующие вопросы:
     1. Какая модель справилась лучше с поставленной задачей?
     2. Имеет ли место переобучение?
     3. Имеет ли место недообучение?
     4. Как можно улучшить метрики моделей?

---
P.S.  
Просьба -- делать каждое задание в отдельных ячейках и с отдельными заголовками (как пункт 1 и 2 в этом ноутбуке) типа  
- Заголовок
- Ячейки с кодом
- Другой заголовок
- Другие ячейки с кодом

P.S.S.  
Если вам повезло с многоклассовой классификацией, у вас не будет проблем, просто нужно будет поставить необходимое количество нейронов на выходе вашей нейросети.

## 1. Пример импорта данных. Грузим данные

In [2]:
import pandas as pd
data = pd.read_csv("/Users/cene/homework_ai/data/student-por.csv")

## 2. Понимаем, какая перед нами задача

In [3]:
'''
G3 - final grade (numeric: from 0 to 20, output target)
значит, что можно сначала решить регрессией и классификацией, но регрессией будет лучше
'''

'\nG3 - final grade (numeric: from 0 to 20, output target)\nзначит, что можно сначала решить регрессией и классификацией, но регрессией будет лучше\n'

## 3. Делаем предобработку данных

In [4]:
# train_test_split
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

data_object = data.select_dtypes(include = ['object'])

impute = SimpleImputer(strategy = 'most_frequent')
job = impute.fit_transform(data_object[['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob',
       'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities',
       'nursery', 'higher', 'internet', 'romantic']])

job = pd.DataFrame(job)
ohec = OneHotEncoder(sparse_output=False)
ohec.fit_transform(job)
ohec.categories_
encoded_job = pd.DataFrame(ohec.fit_transform(job), columns=ohec.get_feature_names_out(['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob',
       'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities',
       'nursery', 'higher', 'internet', 'romantic']))
data_int = data.select_dtypes(include=['int64'])
data_new = pd.concat([encoded_job, data_int], axis=1)
X1 = data_new.drop(columns = ['G3', 'G1', 'G2'])
y1 = data_new["G3"]
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size =0.2, random_state=655) 



In [6]:
from sklearn.preprocessing import StandardScaler
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
scaler = StandardScaler()
scaler.fit(X1_train)
transformed_X1_train = scaler.transform(X1_train)
transformed_X1_test = scaler.transform(X1_test)

In [11]:
transformed_X1_train.shape, y1_train.shape

((519, 56), (519,))

## 4. Dataset

In [22]:
from torch.utils.data import Dataset, DataLoader
import torch

class MyDataset(Dataset):
    def __init__(self, X, y):

        X = torch.Tensor(X)
        y = torch.Tensor(y.values)
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [23]:
train_dataloader = DataLoader(MyDataset(transformed_X1_train, y1_train), batch_size=4)
test_dataloader = DataLoader(MyDataset(transformed_X1_test, y1_train), batch_size=4)

## 5 Train

In [26]:
from torch import nn
class SimpleNet(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.fc(x)

class MediumNet(nn.Module):
    def __init__(self, input_size, hidden_size=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
    
    def forward(self, x):
        return self.net(x)

class DeepNet(nn.Module):
    def __init__(self, input_size, hidden_size=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size // 2, hidden_size // 4),
            nn.ReLU(),
            nn.Linear(hidden_size // 4, 1)
        )
    
    def forward(self, x):
        return self.net(x)

In [ ]:
def train_model(model, optimizer, epochs=5, name="Model"):
    criterion = nn.L1Loss()
    train_losses = []
    test_losses = []
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for X_batch, y_batch in train_dataloader:
            optimizer.zero_grad()
            preds = model(X_batch)
            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        train_losses.append(epoch_loss / len(train_dataloader))
        
        model.eval()
        with torch.no_grad():
            test_loss = 0
            for X_batch, y_batch in test_dataloader:
                preds = model(X_batch)
                test_loss += criterion(preds, y_batch).item()
            test_losses.append(test_loss / len(test_dataloader))
        
        if (epoch+1) % 20 == 0:
            print(f"{name} | Epoch {epoch+1}/{epochs} | Train MAE: {train_losses[-1]:.4f} | Test MAE: {test_losses[-1]:.4f}")
    
    return train_losses, test_losses

In [36]:
import torch.optim as optim
input_size = transformed_X1_train.shape[1]

model1 = SimpleNet(input_size)
opt1 = optim.SGD(model1.parameters(), lr=0.01)
print("Обучаем SimpleNet (SGD)...")
train1, test1 = train_model(model1, opt1, epochs=100, name="SimpleNet (SGD)")

model2 = MediumNet(input_size)
opt2 = optim.Adam(model2.parameters(), lr=0.01)
print("\nОбучаем MediumNet (Adam)...")
train2, test2 = train_model(model2, opt2, epochs=150, name="MediumNet (Adam)")

model3 = DeepNet(input_size)
opt3 = optim.Adam(model3.parameters(), lr=0.003)
print("\nОбучаем DeepNet (Adam)...")
train3, test3 = train_model(model3, opt3, epochs=200, name="DeepNet (Adam)")

Обучаем SimpleNet (SGD)...
SimpleNet (SGD) | Epoch 20/100 | Train MAE: 2.3933 | Test MAE: 2.7239
SimpleNet (SGD) | Epoch 40/100 | Train MAE: 2.3903 | Test MAE: 2.7467
SimpleNet (SGD) | Epoch 60/100 | Train MAE: 2.3906 | Test MAE: 2.7507
SimpleNet (SGD) | Epoch 80/100 | Train MAE: 2.3909 | Test MAE: 2.7512
SimpleNet (SGD) | Epoch 100/100 | Train MAE: 2.3907 | Test MAE: 2.7486

Обучаем MediumNet (Adam)...
MediumNet (Adam) | Epoch 20/150 | Train MAE: 2.3497 | Test MAE: 3.2075
MediumNet (Adam) | Epoch 40/150 | Train MAE: 2.2945 | Test MAE: 3.0770
MediumNet (Adam) | Epoch 60/150 | Train MAE: 2.2427 | Test MAE: 3.2625
MediumNet (Adam) | Epoch 80/150 | Train MAE: 2.2017 | Test MAE: 3.1149
MediumNet (Adam) | Epoch 100/150 | Train MAE: 2.2126 | Test MAE: 2.9511
MediumNet (Adam) | Epoch 120/150 | Train MAE: 2.1896 | Test MAE: 2.9686
MediumNet (Adam) | Epoch 140/150 | Train MAE: 2.1639 | Test MAE: 2.9393

Обучаем DeepNet (Adam)...
DeepNet (Adam) | Epoch 20/200 | Train MAE: 2.4851 | Test MAE: 3.22

In [ ]:
# лучше всего спраивлась маленькая сетка